In [13]:
from counterfactuals.explainers import Cadex, Fimap
from counterfactuals.visualization import compare
from data import GermanData, AdultData
from tensorflow import keras
import numpy as np

In [14]:
german_data = GermanData('data/datasets/input_german.csv', 'data/datasets/labels_german.csv')
model = keras.models.load_model('models/model_german')
test_X = german_data.X_test.iloc[6]
cadex = Cadex(model)
cf = cadex.generate(test_X)

c:\Users\macie\PycharmProjects\counterfactuals\venv\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\macie\PycharmProjects\counterfactuals\venv\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [38]:
german_data.X_train.head()

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
867.0,-0.743495,0.022856,-0.853567,1.054482,0.582147,-0.699136,-0.426244,-0.613951,-0.600453,-0.252646,...,-0.490214,0.810133,-0.810133,-0.193942,0.193942,-0.747967,-0.432370,-0.456573,1.743675,-0.260942
829.0,2.345464,1.205381,0.938700,1.054482,0.229197,-0.699136,2.346074,1.628793,-0.600453,-0.252646,...,-0.490214,-1.234366,1.234366,-0.193942,0.193942,1.336958,-0.432370,-0.456573,-0.573501,-0.260942
73.0,1.830637,0.948755,-0.853567,-1.649318,0.493909,1.087794,-0.426244,1.628793,-0.600453,-0.252646,...,2.039924,0.810133,-0.810133,-0.193942,0.193942,-0.747967,2.312835,-0.456573,-0.573501,-0.260942
642.0,-0.486082,0.478922,0.042566,-0.748051,0.405672,-0.699136,-0.426244,-0.613951,-0.600453,-0.252646,...,-0.490214,-1.234366,1.234366,-0.193942,0.193942,1.336958,-0.432370,-0.456573,-0.573501,-0.260942
270.0,-0.228669,-0.213296,0.938700,0.153215,-0.300228,-0.699136,-0.426244,-0.613951,-0.600453,-0.252646,...,-0.490214,0.810133,-0.810133,5.156181,-5.156181,-0.747967,2.312835,-0.456573,-0.573501,-0.260942


In [15]:
df = compare(test_X, cf.iloc[0])
df

,X,X',change
job_unemployed/unskilled - non-resident,-0.151717,-0.423436,-0.271719
foreign_yes,0.193942,0.464533,0.270591
foreign_no,-0.193942,-0.460756,-0.266814
savings_>= 1000 DM,-0.226381,-0.491633,-0.265252
sex_status_male: divorced/separated,-0.235391,-0.501740,-0.266349
...,...,...,...
purpose_domestic appliances,-0.118075,0.099494,0.217569
purpose_repairs,-0.142857,0.126915,0.269772
purpose_others,-0.118075,0.153820,0.271895
purpose_retraining,-0.079305,0.185395,0.264700


In [16]:
test_X = german_data.input.iloc[6]
cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)
cf = cadex.generate(test_X)
if cf:
    compare(test_X, cf.iloc[0])
else:
    print("cf not generated")

cf not generated


In [17]:
test_X = german_data.X_test.iloc[10]
cadex = Cadex(model, n_changed=8)
cf = cadex.generate(test_X)
compare(test_X, cf.iloc[0])

,X,X',change
purpose_retraining,-0.079305,-0.151227,-0.071921
purpose_domestic appliances,-0.118075,-0.189994,-0.071919
purpose_others,-0.118075,-0.189939,-0.071864
purpose_repairs,-0.142857,-0.214614,-0.071757
credit_history_no credits taken,-0.193942,-0.265863,-0.071921
...,...,...,...
savings_>= 1000 DM,-0.226381,-0.154461,0.071920
age,-0.211991,-0.140070,0.071921
foreign_yes,0.193942,0.122022,-0.071920
job_unemployed/unskilled - non-resident,-0.151717,-0.079796,0.071921


In [22]:
model = keras.models.load_model('models/model_german')
model_predictions = model.predict(german_data.X_train)
model_predictions = np.argmax(model_predictions, axis=1)
fimap = Fimap()
fimap.fit(german_data.X_train, model_predictions)

test_X = german_data.X_test.iloc[0]
cf = fimap.generate(test_X)

Epoch 1/5
25/25 [==============================] - 0s 1ms/step - loss: 0.5174 - binary_accuracy: 0.7625
Epoch 2/5
25/25 [==============================] - 0s 1ms/step - loss: 0.2977 - binary_accuracy: 0.8675
Epoch 3/5
25/25 [==============================] - 0s 1ms/step - loss: 0.2168 - binary_accuracy: 0.9062
Epoch 4/5
25/25 [==============================] - 0s 1ms/step - loss: 0.1824 - binary_accuracy: 0.9262
Epoch 5/5
25/25 [==============================] - 0s 1ms/step - loss: 0.1535 - binary_accuracy: 0.9375
Epoch 1/5
25/25 [==============================] - 0s 1ms/step - loss: 1.0902 - binary_accuracy: 0.6625
Epoch 2/5
25/25 [==============================] - 0s 1ms/step - loss: 0.5320 - binary_accuracy: 0.7412
Epoch 3/5
25/25 [==============================] - 0s 1ms/step - loss: 0.4317 - binary_accuracy: 0.7975
Epoch 4/5
25/25 [==============================] - 0s 1ms/step - loss: 0.3682 - binary_accuracy: 0.8450
Epoch 5/5
25/25 [==============================] - 0s 1ms/step -

In [23]:
df = compare(test_X, cf.iloc[0])
df

,X,X',change
job_unemployed/unskilled - non-resident,-0.151717,-0.319465,-0.167748
employment_4..7 years,-0.432370,-0.635050,-0.202680
debtors_guarantor,-0.232419,-0.301150,-0.068731
property_building society savings agreement,-0.544607,-0.700872,-0.156266
credit_history_delay in paying off in the past,-0.307232,-0.388503,-0.081271
...,...,...,...
foreign_yes,0.193942,-0.356254,-0.550196
purpose_car (used),-0.337944,0.621238,0.959182
purpose_domestic appliances,-0.118075,0.219810,0.337885
debtors_co-applicant,-0.210681,0.502541,0.713222


In [33]:
from counterfactuals.explainers import ECE

ece = ECE(5, columns=list(german_data.X_train.columns), bces=[cadex, fimap], dist=2, h=8, lambda_=0.2, n_jobs=1)

cfs = ece.generate(german_data.X_test.iloc[0])


C:\Users\macie\AppData\Local\Temp/ipykernel_21160/3035201521.py:5: UserWarning: k parameter > number of aggregated counterfactuals. Changing k from 5 to 2
  cfs = ece.generate(german_data.X_test.iloc[0])
C:\Users\macie\AppData\Local\Temp/ipykernel_21160/3035201521.py:5: UserWarning: knn's h parameter >= number of aggregated counterfactuals. Changing h from  8 to 1
  cfs = ece.generate(german_data.X_test.iloc[0])


In [34]:
cfs

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,1.025951,-0.470915,0.653647,0.767822,-0.669506,-0.546905,-0.708093,-0.895096,1.377618,-0.039027,...,-0.725711,0.544883,-0.529304,0.093881,-0.093712,-0.460121,-0.143089,-0.202748,2.032283,-0.534292


In [35]:
for i in range(cfs.shape[0]):
    compare(german_data.X_test.iloc[0], cfs.iloc[i, :])

In [36]:
ece.get_aggregated_cfs()

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,0.142731,0.114774,0.162764,-0.100209,0.006560,-0.350128,0.242996,-0.570087,0.106077,0.062363,...,-0.129596,0.459505,-0.313161,0.261140,-0.356254,-0.164691,-0.635050,-0.560369,0.187094,0.451313
1,1.025951,-0.470915,0.653647,0.767822,-0.669506,-0.546905,-0.708093,-0.895096,1.377618,-0.039027,...,-0.725711,0.544883,-0.529304,0.093881,-0.093712,-0.460121,-0.143089,-0.202748,2.032283,-0.534292


In [ ]:
german_data.X_train.iloc[0]